In [ ]:
import sys, os
import pandas as pd
import netCDF4 as nc
from datetime import datetime, timezone
import numpy as np
sys.path.append('../src/')
from Biologging_Toolkit.applications.Mixed_Layer_Depth import MixedLayerDepth
from Biologging_Toolkit.processing.Dives import Dives
from Biologging_Toolkit.utils.format_utils import get_start_time_sens

In [ ]:
depid = 'ml18_294b'
path = os.path.join('D:/individus_brut/individus/', depid)
ref_path = os.path.join(path, 'data', 'auxiliary', 'instrument')
sens_path = os.path.join(ref_path, depid+'sens5.nc')

### Make sure csv structure for dive data exists

In [ ]:
dive = Dives(depid, path = ref_path, sens_path = sens_path)

In [ ]:
dive()

### Add temperature data to reference structure

In [ ]:
ds = nc.Dataset(sens_path)
temperature = ds['T'][:].data
temp_time = get_start_time_sens(ds.dephist_device_datetime_start) + np.arange(0, len(temperature))/5

In [ ]:
dive.create_variable('temperature',
                     var_data =  temperature,
                     var_time = temp_time)

In [ ]:
dive.ds

In [ ]:
dive.ds.close()

### Compute mixed layer depth

In [ ]:
inst = MixedLayerDepth(depid, 
            path = ref_path
           )

In [ ]:
inst()

### Wind correlation with MLD

In [ ]:
depids = ['ml18_296a','ml18_294b','ml19_292a','ml19_292b','ml19_293a','ml19_294a','ml20_293a','ml20_296b','ml20_313a','ml21_295a','ml21_305b','ml17_280a']
#path = '/run/media/grosmaan/LaCie/individus_brut/individus/'
path = 'D:/individus_brut/individus/'
paths = [os.path.join(path, depid) for depid in depids]

In [ ]:
from Biologging_Toolkit.applications.Wind import Wind
from Biologging_Toolkit.utils.plot_utils import subplots_centered
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib import colormaps
import seaborn as sns
plt.rcParams.update({
    "text.usetex": True,                # Enable LaTeX text rendering
    "font.family": "serif",             # Use a serif font
    "font.serif": ["Computer Modern"],  # Set font to Computer Modern (LaTeX default)
})
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
corrected_mld = False
depids_with_mld = []
for depid in depids :
    df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
    try :
        if np.all(np.isnan(df.meop_mld.to_numpy())):
            continue
        depids_with_mld.append(depid)
    except AttributeError:
        continue
if corrected_mld :
    depids_with_mld = []
    for depid in depids :
        df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
        if 'corr_mld' in list(df.columns) :
            depids_with_mld.append(depid)
print(depids_with_mld)

In [ ]:
def get_previous_wind_single(df, data, groupby = 'FOD') :
    wind_interp = interp1d(df.end_time, df[data], bounds_error = False)
    for i in range(48):
        df[f'wind_{i}h'] = wind_interp(df.end_time - i*3600)
    if groupby :
        data = ['meop_mld', groupby]
    else :
        data = ['meop_mld']
    for i in range(48):
        data.append(f'wind_{i}h')
    return df[data]

In [ ]:
def get_previous_wind(df, data) :
    wind_interp = interp1d(df.end_time, df.wind_speed, bounds_error = False)
    pensieri_interp = interp1d(df.end_time, df.pensieri, bounds_error = False)
    lstm_interp = interp1d(df.end_time, df.lstm, bounds_error = False)
    hild_interp = interp1d(df.end_time, df.hildebrand, bounds_error = False)
    for i in range(48):
        df[f'wind_{i}h'] = wind_interp(df.end_time - i*3600)
        df[f'pensieri_{i}h'] = pensieri_interp(df.end_time - i*3600)
        df[f'lstm_{i}h'] = lstm_interp(df.end_time - i*3600)
        df[f'hildebrand_{i}h'] = hild_interp(df.end_time - i*3600)
    return df

In [ ]:
def get_correlation(depids, depid, path, data = 'downwards_mean_5000', FOD = 'FOD') :
    df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
    inst = Wind(depids, path = paths, test_depid = depid, data = data)
    inst.depid_fit()
    est = inst.df[inst.df.depid == depid].depid_estimation.to_numpy()
    df['pensieri'] = est
    inst = Wind(depids, path = paths, test_depid = depid, data = data, method = 'Hildebrand')
    inst.depid_fit()
    est = inst.df[inst.df.depid == depid].depid_estimation.to_numpy()
    df['hildebrand'] = est
    df = get_previous_wind(df, data = data)
    if FOD :
        data_pens = ['meop_mld', FOD]; data_era = ['meop_mld', FOD]; data_lstm = ['meop_mld', FOD]; data_hild = ['meop_mld', FOD]
    else :
        data_pens = ['meop_mld']; data_era = ['meop_mld']; data_lstm = ['meop_mld']; data_hild = ['meop_mld']
    for i in range(48):
        data_pens.append(f'pensieri_{i}h')
        data_era.append(f'wind_{i}h')
        data_lstm.append(f'lstm_{i}h')
        data_hild.append(f'hildebrand_{i}h')
    return df[data_era], df[data_pens], df[data_lstm], df[data_hild]

In [ ]:
#### HEATMAP FOR BINNED INDEPENDANT VARIABLE OF CORRELATION AS A FUNCTION OF WIND AVERAGING PERIOD AND HOURS BEFORE MLD
colors = colormaps.get_cmap('plasma').resampled(5)
FOD = True
fod_labels = [1,2,3,4,5]

df = pd.DataFrame()
for i, depid in enumerate(depids_with_mld) :
    _df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
    _df = get_previous_wind_single(_df, 'lstm', groupby = 'temp_200m')
    for avg in range(2, 25) :
        _roll = _df.iloc[:, 2:50].rolling(avg, min_periods=1, center=True).mean()
        _roll.columns = [f'avg{avg}_{col}' for col in list(_roll.columns)]
        _df = pd.concat((_df, _roll), axis = 1)
    df = pd.concat((df, _df))
    
temp_bins = [0, 3, 6, 12.5, float('inf')]
mld_bins = [0, df['meop_mld'].quantile(0.33), df['meop_mld'].quantile(0.66), np.inf]
temp_bins = [0, df['temp_200m'].quantile(0.25), df['temp_200m'].quantile(0.5), df['temp_200m'].quantile(0.75), np.inf]
labels = [1, 2, 3, 4]
df['bins'] = pd.cut(df['temp_200m'], bins=temp_bins, labels=labels, right=False)

if FOD:
    fig, ax = plt.subplots(2,2,figsize = (12, 12), sharey = True)
    ax = ax.flatten()
    for k in range(1, 5):
        sns.heatmap(
            df[df.bins == k].corr()['meop_mld'][2:-1].to_numpy().reshape(-1, 48), 
            ax=ax[k - 1],
            cbar_kws={'label': 'Pearson correlation coefficient'}
        )
        ax[k-1].set_title(f'Bin {k}')
else :
    fig, ax = plt.subplots(figsize = (15, 10))
    sns.heatmap(
            df.corr()['meop_mld'][2:-1].to_numpy().reshape(-1, 48), 
            ax=ax,
            cbar_kws={'label': 'Pearson correlation coefficient'}
        )
fig.text(0.5, 0.04, 'Hours before MLD obtention', ha='center', va='center', fontsize=14)
fig.text(0.04, 0.5, 'Wind averaging period in hours', ha='center', va='center', rotation='vertical', fontsize=14)
fig.tight_layout(rect=[0.05, 0.05, 1, 1])

In [ ]:
fig.savefig('C:/Users/grosm/Desktop/thèse/Figures/wind_averaged_correlation_mld_all_SES.pdf')

In [ ]:
### CORRELATION FOR EACH INDIVIDUAL AS A FUNCTION OF TIME DIFFERENTIAL AND FOR DIFFERENT MODELS

fig, ax = subplots_centered(3,4,figsize = (20, 20), sharey = True, nfigs = 11)
#fig1, ax1 = subplots_centered(3,2,figsize = (10, 10), sharey = True, nfigs = 5)
colors = colormaps.get_cmap('plasma').resampled(5)
FOD = False
df_era = pd.DataFrame()
df_lstm = pd.DataFrame()
df_hild = pd.DataFrame()
df_pens = pd.DataFrame()
for i, depid in enumerate(depids_with_mld) :
    fod_counts = np.array([0,0,0,0,0])
    fod_labels = [1,2,3,4,5]
    _df_era, _df_pens, _df_lstm, _df_hild = get_correlation(depids, depid, path, data = 'upwards_mean_5000', FOD = 'temp_200m')
    df_era = pd.concat((df_era, _df_era))
    df_lstm = pd.concat((df_lstm, _df_lstm))
    df_pens = pd.concat((df_pens, _df_pens))
    df_hild = pd.concat((df_hild, _df_hild))

    #ax1[i].bar(fod_labels, fod_counts, align='center')
    #ax1[i].set_title(depid)
    #ax1[i].grid()
    if FOD :
        labels, counts = np.unique(_df_era.FOD, return_counts=True)
        counts = counts[~np.isnan(labels)]
        labels = labels[~np.isnan(labels)]
        fod_counts[labels.astype(int)-1] = counts
        for k in range(1,6) :
            ax[i].plot(_df_era[_df_era.FOD == k].corr()['meop_mld'][2:].to_numpy(), color = colors(k-1), marker = 's')
        #ax[i].plot(np.nanmean(np.column_stack([df_pens[df_pens.FOD == j].corr()['meop_mld'][2:] for j in range(1,2)]), axis = 1), color = colors(0), marker = '^')
        #ax[i].plot(np.nanmean(np.column_stack([df_hild[df_hild.FOD == j].corr()['meop_mld'][2:] for j in range(1,2)]), axis = 1), color = colors(0), marker = 'P')
        #ax[i].plot(np.nanmean(np.column_stack([df_lstm[df_lstm.FOD == j].corr()['meop_mld'][2:] for j in range(1,2)]), axis = 1), color = colors(2), marker = 'o')
    else :
        ax[i].plot(_df_era.corr()['meop_mld'].to_numpy()[2:], color = colors(3))
        ax[i].plot(_df_pens.corr()['meop_mld'].to_numpy()[2:], color = colors(0))
        ax[i].plot(_df_hild.corr()['meop_mld'].to_numpy()[2:], color = colors(1))
        ax[i].plot(_df_lstm.corr()['meop_mld'].to_numpy()[2:], color = colors(2))
    ax[i].grid()
    ax[i].set_title(depid)
legend_handles = [
    plt.Line2D([0], [0], color=colors(0), lw=2, label="Quadratic"),
    plt.Line2D([0], [0], color=colors(1), lw=2, label="Logarithmic"),
    plt.Line2D([0], [0], color=colors(2), lw=2, label="LSTM"),
    plt.Line2D([0], [0], color=colors(3), lw=2, label="ERA"),
    ]
ax[0].legend(handles = legend_handles)
ax[0].set_xticklabels([])
ax[1].set_xticklabels([])
ax[2].set_xticklabels([])
#ax1[0].set_xticklabels([])
#ax1[1].set_xticklabels([])
#ax1[2].set_xticklabels([])
fig.text(0.56, 0.04, 'Hours before MLD obtention', ha='center', va='center', fontsize=14)
fig.text(0.04, 0.5, r'Pearson correlation coefficient with previous wind speeds and MLD', ha='center', va='center', rotation='vertical', fontsize=14)
fig.tight_layout(rect=[0.05, 0.05, 1, 1])
#fig1.text(0.56, 0.04, 'FOD', ha='center', va='center', fontsize=14)
#fig1.text(0.04, 0.5, 'Number of occurences', ha='center', va='center', rotation='vertical', fontsize=14)
#fig1.tight_layout(rect=[0.05, 0.05, 1, 1])


In [ ]:
fig.savefig('C:/Users/grosm/Desktop/thèse/Figures/wind_mld_all_SES.pdf')
#fig1.savefig('C:/Users/grosm/Desktop/thèse/Figures/FOD_histogram.pdf')

In [ ]:
### CORRELATION AVERAGED OVER INDIVIDUALS FOR DIFFERENT MODELS AND BINNED INDEPENDANT VARIABLE

fig, ax = plt.subplots(3,2, figsize = (8,12), sharey = True)
ax = ax.flatten()
colors = colormaps.get_cmap('viridis').resampled(4)
groupby = 'bins'
if groupby == 'bins': 
    nbins = 6
    nbins += 1
    df_lstm['bins'] = pd.cut(df_lstm['temp_200m'], bins=np.linspace(-2, 15, nbins), labels = list(range(1,nbins)), right=False)
    df_era['bins'] = pd.cut(df_era['temp_200m'], bins=np.linspace(-2, 15, nbins), labels = list(range(1,nbins)), right=False)
    df_pens['bins'] = pd.cut(df_pens['temp_200m'], bins=np.linspace(-2, 15, nbins), labels = list(range(1,nbins)), right=False)
    df_hild['bins'] = pd.cut(df_hild['temp_200m'], bins=np.linspace(-2, 15, nbins), labels = list(range(1,nbins)), right=False)
for k in range(1,nbins) :
    ax[k-1].set_title(f"Temperature [{np.linspace(-2, 15, nbins)[k-1]}° : {np.linspace(-2, 15, nbins)[k]}°]")
    ax[k-1].grid()
    ax[k-1].plot(df_era[df_era[groupby] == k].corr()['meop_mld'].to_numpy()[2:], color = colors(0), marker = 's', markevery=(0,5))
    ax[k-1].plot(df_pens[df_pens[groupby] == k].corr()['meop_mld'].to_numpy()[2:], color = colors(1), marker = '^', markevery=(1,5))
    ax[k-1].plot(df_hild[df_hild[groupby] == k].corr()['meop_mld'].to_numpy()[2:], color = colors(2), marker = 'P', markevery=(2,5))
    ax[k-1].plot(df_lstm[df_lstm[groupby] == k].corr()['meop_mld'].to_numpy()[2:], color = colors(3), marker = 'o', markevery=(3,5))
    ax[k-1].plot([np.nanargmax(df_era[df_era[groupby] == k].corr()['meop_mld'].to_numpy()[2:])]*10, 
                 np.linspace(0, np.nanmax(df_era[df_era[groupby] == k].corr()['meop_mld'].to_numpy()[2:]), 10),
                 '--', color = colors(0))
    ax[k-1].plot([np.nanargmax(df_pens[df_pens[groupby] == k].corr()['meop_mld'].to_numpy()[2:])]*10, 
             np.linspace(0, np.nanmax(df_pens[df_pens[groupby] == k].corr()['meop_mld'].to_numpy()[2:]), 10),
             '--', color = colors(1))
    ax[k-1].plot([np.nanargmax(df_hild[df_hild[groupby] == k].corr()['meop_mld'].to_numpy()[2:])]*10, 
                 np.linspace(0, np.nanmax(df_hild[df_hild[groupby] == k].corr()['meop_mld'].to_numpy()[2:]), 10),
                 '--', color = colors(2))
    ax[k-1].plot([np.nanargmax(df_lstm[df_lstm[groupby] == k].corr()['meop_mld'].to_numpy()[2:])]*10, 
                 np.linspace(0, np.nanmax(df_lstm[df_lstm[groupby] == k].corr()['meop_mld'].to_numpy()[2:]), 10),
                 '--', color = colors(3))
legend_handles = [
    plt.Line2D([0], [0], color=colors(0), lw=2, label="ERA5"),
    plt.Line2D([0], [0], color=colors(1), lw=2, label="Quadratic model"),
    plt.Line2D([0], [0], color=colors(2), lw=2, label="Logarithmic model"),
    plt.Line2D([0], [0], color=colors(3), lw=2, label="LSTM inertial")
    ]
ax[0].legend(handles = legend_handles)
fig.text(0.56, 0.04, 'Hours before MLD obtention', ha='center', va='center', fontsize=14)
fig.text(0.04, 0.5, r'Pearson correlation coefficient with previous wind speeds and MLD', ha='center', va='center', rotation='vertical', fontsize=14)
fig.tight_layout(rect=[0.05, 0.05, 1, 1])

In [ ]:
fig.savefig('C:/Users/grosm/Desktop/thèse/Figures/wind_mld_per_temperature_bin.pdf')

In [ ]:
#### GLOBAL CORRELATION AVERAGED OVER ALL ELEPHANT SEALS FOR DIFFERENT WIND MODELS

fig, ax = plt.subplots(figsize = (7,7))
colors = colormaps.get_cmap('viridis').resampled(4)
ax.set_title(f"Correlation for all FODs")
ax.grid()
ax.plot(df_era.corr()['meop_mld'].to_numpy()[2:-1], color = colors(0), marker = 's')
ax.plot(df_pens.corr()['meop_mld'].to_numpy()[2:-1], color = colors(1), marker = '^')
ax.plot(df_hild.corr()['meop_mld'].to_numpy()[2:-1], color = colors(2), marker = 'P')
ax.plot(df_lstm.corr()['meop_mld'].to_numpy()[2:-1], color = colors(3), marker = 'o')
legend_handles = [
    plt.Line2D([0], [0], color=colors(0), lw=2, label="ERA5"),
    plt.Line2D([0], [0], color=colors(1), lw=2, label="Quadratic model"),
    plt.Line2D([0], [0], color=colors(2), lw=2, label="Logarithmic model"),
    plt.Line2D([0], [0], color=colors(3), lw=2, label="LSTM inertial")
    ]
ax.legend(handles = legend_handles)
fig.text(0.56, 0.04, 'Hours before MLD obtention', ha='center', va='center', fontsize=14)
fig.text(0.04, 0.5, r'Pearson correlation coefficient with previous wind speeds and MLD', ha='center', va='center', rotation='vertical', fontsize=14)
fig.tight_layout(rect=[0.05, 0.05, 1, 1])

In [ ]:
fig.savefig('C:/Users/grosm/Desktop/thèse/Figures/global_correlation_SES_with_mld.pdf')

### Wind vs. MLD

In [ ]:
depids = ['ml18_296a','ml18_294b','ml19_292a','ml19_292b','ml19_293a','ml19_294a','ml20_293a','ml20_296b','ml20_313a','ml21_295a','ml21_305b','ml17_280a']
path = 'D:/individus_brut/individus/'
paths = [os.path.join(path, depid) for depid in depids]

In [ ]:
from Biologging_Toolkit.applications.Wind import Wind
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams.update({
    "text.usetex": True,                # Enable LaTeX text rendering
    "font.family": "serif",             # Use a serif font
    "font.serif": ["Computer Modern"],  # Set font to Computer Modern (LaTeX default)
})
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
def get_previous_state(df, data, i) :
    wind_interp = interp1d(df.end_time, df[data], bounds_error = False)
    mld_interp = interp1d(df.end_time, df.meop_mld, bounds_error = False)
    #df[f'{data}_{i}h'] = wind_interp(df.end_time - i*3600)
    df[f'{data}_{i}h'] = wind_interp(df.end_time - i*3600)
    df[f'mld_{i}h'] = mld_interp(df.end_time - i*3600)
    return df

In [ ]:
def get_corr_df(depids, depid, path, data = 'upwards_mean_5000', i = 15) :
    df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
    '''inst = Wind(depids, path = paths, test_depid = depid, data = data)
    inst.depid_fit()
    est = inst.df[inst.df.depid == depid].depid_estimation.to_numpy()
    df['temp_model'] = est'''
    df = get_previous_state(df, data = 'lstm', i = i)
    df['mld_variation'] = df.meop_mld - df[f'mld_{i}h']
    df['wind_variation'] = df.lstm - df[f'lstm_{i}h']
    return df

In [ ]:
### SHOW MLD VARIATION AS A FUNCTION OF WIND SPEED FOR DIFFERENT FODs
fig, ax = subplots_centered(3, 2, figsize = (13, 13), sharex = True, sharey = True, nfigs = 5)
hour_fod = [47,10,35,22,11]
FOD_df = pd.DataFrame()
for fod in range(1,6) :
    df = pd.DataFrame()
    for j, depid in enumerate(['ml18_294b','ml19_292b','ml19_293a','ml19_294a','ml20_296b']) :
        _df = get_corr_df(depids, depid, path, data = 'downwards_mean_5000', i = hour_fod[fod-1])
        df = pd.concat((df, _df[['FOD', 'mld_variation', 'lstm', 'max_depth', 'meop_mld']]))
    df.reset_index(inplace = True), 
    df[df.lstm < 5] = np.nan
    df = df[df.FOD == fod]
    FOD_df = pd.concat((FOD_df, df))
    sns.kdeplot(df, x = 'lstm', y='mld_variation', ax = ax[fod-1])
    sns.regplot(df, x = 'lstm', y='mld_variation', scatter = False, lowess = True, ax = ax[fod-1])
    #sns.regplot(df, x = 'LSTM', y='mld_variation', scatter = False, order = 2, ax = ax[fod-1])
    ax[fod-1].grid()
    ax[fod-1].set_xlabel('')
    ax[fod-1].set_ylabel('')
    ax[fod-1].set_ylim(-100, 200)
    ax[fod-1].set_title(f"FOD {fod}")
fig.text(0.56, 0.04, 'Wind speed (m/s)', ha='center', va='center', fontsize=14)
fig.text(0.04, 0.55, 'MLD variation (m)', ha='center', va='center', rotation='vertical', fontsize=14)
fig.tight_layout(rect=[0.05, 0.05, 1, 1])

In [ ]:
fig.savefig('C:/Users/grosm/Desktop/thèse/Figures/wind_impact_on_mld.pdf')

In [ ]:
### GLOBAL MLD VARIATION AFTER CHOSEN TIME LAG
fig, ax = plt.subplots(1,1, figsize = (10, 10))
df = pd.DataFrame()
for j, depid in enumerate(depids_with_mld) :
    _df = get_corr_df(depids, depid, path, data = 'downwards_mean_5000', i = 15)
    df = pd.concat((df, _df[['mld_variation', 'lstm_15h', 'max_depth', 'meop_mld']]))
df = df.reset_index()
ax.set_title('MLD variation after 15 hours and initial MLD')
ax.scatter(df.meop_mld, df.mld_variation,
                  color="orange", alpha=0.1)
ax.plot(np.linspace(0, 400, 100), 0.97*np.linspace(0, 400, 100) - 10.67, c = 'red')
ax.set_xlabel('MLD (m)')
ax.set_ylabel('MLD variation (m)')

In [ ]:
fig.savefig('C:/Users/grosm/Desktop/thèse/Figures/mld_variation.pdf')

### Wind gust duration

In [ ]:
from scipy.signal import find_peaks, medfilt
from scipy.optimize import curve_fit
from scipy.ndimage import generic_filter
from scipy import odr
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.families import Gaussian
import statsmodels.genmod.bayes_mixed_glm as bayes_mixed_glm
from statsmodels.multivariate.pca import PCA
def norm(x) :
    return (x - np.nanmin(x)) / (np.nanmax(x) - np.nanmin(x))
from Biologging_Toolkit.utils.inertial_utils import coa
from Biologging_Toolkit.utils.format_utils import numpy_fill

In [ ]:
def get_peaks(df, data = 'lstm', threshold = 10, time_diff = 15) :
    data = df[data].to_numpy()
    timeframe = df.begin_time.to_numpy()
    transition = np.diff((numpy_fill(data) >= threshold).astype(int))
    first_peak = np.where((transition == 1))[0][0]
    transition[:first_peak] = 0
    left_base = np.where((transition == 1))[0]
    right_base = np.where((transition == -1))[0]
    bases = np.column_stack((left_base[:np.min((len(left_base), len(right_base)))], right_base[:np.min((len(left_base), len(right_base)))]))
    wind_max, wind_mean, duration, peaks = [], [], [], []
    for base in bases :
        if base[1] - base[0] < 2:
            continue
        peaks.append(base[0] + np.argmax(np.nan_to_num(data[base[0]:base[1]])))
        wind_max.append(np.nanmax(data[base[0]:base[1]]))
        wind_mean.append(np.nanmean(data[base[0]:base[1]]))
        duration.append(timeframe[base[1]] - timeframe[base[0]])
    mld = df.meop_mld.to_numpy()
    temp = df.temp_10m.to_numpy()
    gradient = df.gradient.to_numpy()
    mld_data = []
    temp_data = []
    gradient_data = []
    for peak in peaks :
        gradient_data.append(gradient[peak])
        _mld = [mld[peak]]
        _temp = [temp[peak]]
        j = 0
        while (peak+j < len(timeframe)) and (timeframe[peak + j] - timeframe[peak] < time_diff * 3600)  :
            _mld.append(mld[peak + j])
            _temp.append(temp[peak + j])
            j+=1
        mld_data.append(_mld)
        temp_data.append(_temp)
    return np.array(wind_max), np.array(wind_mean), np.array(duration), mld_data, temp_data, np.array(gradient_data)

In [ ]:
def get_wind_gust(df, data = 'lstm', time_diff = 15) :
    peaks, _ = find_peaks(df[data].to_numpy(), prominence = 1.5, height = 6, distance = 15)
    timeframe = df.begin_time.to_numpy()
    begin_gust, end_gust = timeframe[_['left_bases']], timeframe[_['right_bases']]
    duration = end_gust - begin_gust
    max_speed = df[data].to_numpy()[peaks]
    temp20 = df['temp_20m'].to_numpy()[peaks]
    temp10 = df['temp_10m'].to_numpy()[peaks]
    average = []
    for lb, rb in zip(_['left_bases'], _['right_bases']):
        average.append(np.nanmean(df[data].to_numpy()[lb:rb+1]))
    mld = df.meop_mld.to_numpy()
    temp = df.temp_10m.to_numpy()
    gradient = df.gradient.to_numpy()
    mld_data = []
    temp_data = []
    other_peaks = []
    gradient_data = []
    for peak in peaks :
        _other_peak = [np.nan]
        _mld = [mld[peak]]
        _temp = [temp[peak]]
        gradient_data.append(gradient[peak])
        j = 0
        while (peak+j < len(timeframe)) and (timeframe[peak + j] - timeframe[peak] < time_diff * 3600)  :
            _mld.append(mld[peak + j])
            _temp.append(mld[peak + j])
            if (peak+j>peak) and (peak+j in peaks) :
                _other_peak.append(df[data].to_numpy()[peak+j])
            j+=1
        other_peaks.append(len(_other_peak))
        mld_data.append(_mld)
        temp_data.append(_temp)
    return max_speed, np.array(average), duration, mld_data, np.array(other_peaks), temp_data, np.array(gradient_data)

In [ ]:
_df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
data = _df['lstm'].to_numpy()
timeframe = _df.begin_time.to_numpy()
transition = np.diff((numpy_fill(data) >= 10).astype(int))
first_peak = np.where((transition == 1))[0][0]
transition[:first_peak] = 0
left_base = np.where((transition == 1))[0]
right_base = np.where((transition == -1))[0]
bases = np.column_stack((left_base[:np.min((len(left_base), len(right_base)))], right_base[:np.min((len(left_base), len(right_base)))]))
wind_max, wind_mean, duration, peaks = [], [], [], []
for base in bases :
    if base[1] - base[0] < 2:
        continue
    peaks.append(base[0] + np.argmax(np.nan_to_num(data[base[0]:base[1]])))

peaks, _ = find_peaks(data, prominence = 1.5, height = 6, distance = 10)
left_base, right_base = _['left_bases'], _['right_bases']
bases = np.column_stack((left_base[:np.min((len(left_base), len(right_base)))], right_base[:np.min((len(left_base), len(right_base)))]))
fig, ax = plt.subplots(figsize = (15,6))
ax.plot(timeframe, data)
ax.scatter(timeframe[peaks], data[peaks], c = 'orange', s = 25)
for base in bases :
    ax.plot([timeframe[base[0]], timeframe[base[1]]], [10,10], '--o', c = 'red')
ax.set_xlim(timeframe[0], timeframe[500])

In [ ]:
intercept, peaks_coef, duration_coef, p_values, peaks_pval, duration_pval, r_squared, adj_r_squared = [],[],[],[],[],[],[],[]
average_coef, average_pval, pmld_coef, pmld_pval = [],[],[],[]
temp_coef, temp_pval, gradient_coef, gradient_pval = [], [], [], []
intercept_se, peaks_se, duration_se = [], [], []
r_squared_peaks, r_squared_duration = [], []

params = ['peaks', 'duration', 'gradient', 'previous_mld', 'const']
res_values = {par:[[],[]] for par in params}
for j in np.arange(1,48,0.5) :
    all_mld = []
    all_temp = []
    temp_var = []
    mld_var = []
    df = pd.DataFrame()
    for i, depid in enumerate(depids_with_mld) :
        _df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
        peaks, average, duration, mld_data, other_peaks, temp_data, gradient = get_wind_gust(_df, time_diff = j)
        #peaks, average, duration, mld_data, temp_data, gradient = get_peaks(_df, time_diff = j)
        mld, previous_mld = [_mld_data[-1] for _mld_data in mld_data], [_mld_data[0] for _mld_data in mld_data]
        temp = [_temp_data[0] for _temp_data in temp_data]
        _df = pd.DataFrame({'peaks':peaks,
                                          'duration':duration,
                                          'mld':mld,
                                          'previous_mld':previous_mld,
                                          'average':average,
                                          'temp10': temp,
                                          'gradient':gradient})
        _df = _df.dropna(subset=params[:-1]+['mld'])
        all_mld.extend([_mld_data for i,_mld_data in enumerate(mld_data) if i in _df.index])
        all_temp.extend([_temp_data for i,_temp_data in enumerate(temp_data) if i in _df.index])
        temp_var.extend([np.nanvar(_temp_data) for i,_temp_data in enumerate(temp_data) if i in _df.index])
        mld_var.extend([np.nanvar(_mld_data) for i,_mld_data in enumerate(mld_data) if i in _df.index])
        df = pd.concat((df, _df))
    df['var_temp'] = temp_var
    df['var_mld'] = mld_var

    df.reset_index(inplace = True, drop = True)
    df = df[df.var_mld < 5000]
    df['mld_diff'] = df['mld'] - df['previous_mld']
    df = df[df.mld_diff > 0]

    X = df[params[:-1]]
    X = sm.add_constant(X)
    y = df['mld']

    model = sm.OLS(y, X)
    #model = bayes_mixed_glm.BinomialBayesMixedGLM(y, X)
    #data = odr.Data(df.peaks.to_numpy(), df.mld.to_numpy())
    #odr_obj = odr.ODR(data, odr.quadratic)
    #output = odr_obj.run()
    results = model.fit()

    for val in results.params.keys() :
        res_values[val][0].append(results.params[val])
        res_values[val][1].append(results.pvalues[val])
    r_squared.append(results.rsquared)
for val in results.params.keys() :
    res_values[val] = np.array(res_values[val])

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax = ax.flatten()

scat = ax[0].scatter(df.peaks.to_numpy(), df.mld.to_numpy(), c=df.var_mld.to_numpy())
fig.colorbar(scat, ax=ax[0])
ax[0].set_title("Variance MLD")
scat = ax[1].scatter(df.peaks.to_numpy(), df.mld.to_numpy(), c=df.gradient.to_numpy())
fig.colorbar(scat, ax=ax[1])
ax[1].set_title("Variance 10m Temperature")
mask = (df.temp10.to_numpy() < 50) & (df.var_temp < 15500)
scat = ax[2].scatter(df.peaks.to_numpy()[mask], df.mld.to_numpy()[mask], c=df.temp10.to_numpy()[mask], alpha=0.9)
fig.colorbar(scat, ax=ax[2])
sns.kdeplot(df[mask], x='peaks', y='mld', ax=ax[2], alpha = 0.7)
ax[2].set_title(r"10m Temperature at $t_0$")
mask = (df.temp10.to_numpy() <= 50) & (df.var_mld < 15400)
scat = ax[3].scatter(df.peaks.to_numpy()[mask], df.mld.to_numpy()[mask], c=df.duration.to_numpy()[mask])
fig.colorbar(scat, ax=ax[3])
ax[3].set_title("Wind gust duration")

for a in ax:
    a.set_xlabel("")
    a.set_ylabel("")
fig.text(0.5, 0.0, "Wind Speed (m/s)", ha="center", fontsize=12)
fig.text(0.0, 0.5, "MLD (m)", va="center", rotation="vertical", fontsize=12)

fig.tight_layout()
fig.show()

In [ ]:
### PLOT COEFFICIENTS, R2, PVALUES AND MODEL PREDICTION AS A FUNCTION OF TIME LAG

fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax = ax.flatten()
colors = ['blue', 'red', 'fuchsia', 'gold', 'green']
markers = ['o','p','s','x']
labels = {'peaks':'Maximum wind speed', 'duration':'Wind gust duration', 'previous_mld':'Previous MLD', 'gradient':'Density gradient at MLD', 'temp10':'Temperature at 10m'}

lines = []
ax1 = ax[0].twinx()
for i, val in enumerate(res_values.keys()) :
    if val == 'const':
        continue
    if val == 'gradient' :
        line = ax1.plot(res_values[val][0], colors[i], marker = markers[i], label = labels[val], markevery = 2)
        ax1.set_ylabel('Gradient coefficient', color=colors[i])
        ax1.tick_params(axis='y', labelcolor=colors[i])
        lines.extend(line)
        continue
    line = ax[0].plot(res_values[val][0], colors[i], marker = markers[i], label = labels[val], markevery = 2)
    lines.extend(line)

ax0_labels = [l.get_label() for l in lines]
ax[0].legend(lines, ax0_labels, loc='upper left')
ax[0].grid()
ax[0].set_xlabel('Time differential between wind gust and MLD (h)')
ax[0].set_title('Linear regression coefficients')

ax[1].plot(r_squared, color='green', label=r'Global $R^2$', marker='^')
#np.argmax(r_squared)
ax[1].plot([13]*2, [0, r_squared[13]], '--', c = 'k')
ax[1].set_ylabel(r'$R^2$')
ax[1].set_xlabel('Time differential between wind gust and MLD (h)')
ax[1].legend()
ax[1].set_title(r'$R^2$ between dependant variables and $\Delta$MLD')
ax[1].grid()

lines = []
for i, val in enumerate(res_values.keys()) :
    if val == 'const':
        continue
    line = ax[2].plot(res_values[val][1], colors[i], marker = markers[i], label = labels[val], markevery = 2)
    lines.extend(line)

ax[2].set_xlabel('Time differential between wind gust and MLD (h)')
ax[2].set_title('P-values for wind gust speed and duration')
ax[2].grid()

plot_df = pd.DataFrame()
for i, depid in enumerate(depids_with_mld) :
    _plot_df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
    #np.argmax(r_squared)
    peaks, average, duration, mld_data, temp_data, gradient = get_peaks(_plot_df, time_diff = 13)
    #peaks, average, duration, mld_data, other_peaks, temp_data = get_wind_gust(_plot_df, time_diff = 14)
    mld, previous_mld = [_mld_data[-1] for _mld_data in mld_data], [_mld_data[0] for _mld_data in mld_data]
    temp = [_temp_data[0] for _temp_data in temp_data]
    plot_df = pd.concat((plot_df, pd.DataFrame({'peaks':peaks, 
                                      'duration':duration, 
                                      'mld':mld, 
                                      'previous_mld':previous_mld,
                                      'average':average,
                                      'temp10':temp,
                                      'gradient':gradient})))
plot_df.reset_index(inplace = True)
plot_df['pred'] = np.array([res_values[val][0,13]*plot_df[val].to_numpy() for val in list(res_values.keys())[:-1]]).sum(axis = 0) + res_values['const'][0,13]
sns.kdeplot(plot_df, x = 'mld', y = 'pred', ax=ax[3])
ax[3].scatter(plot_df.mld, plot_df.pred, alpha = 0.2, c = 'royalblue')
ax[3].plot([-40,400], [-40,400], '--' , c='k')
ax[3].grid()
ax[3].set_title(r"Model's prediction for 13h lag")
fig.tight_layout()
fig.show()

In [ ]:
fig.savefig('C:/Users/grosm/Desktop/thèse/Figures/OLS1_peaks_duration_pmld_average.pdf')

In [ ]:
### GET FINAL LAW DESCRIBING BEHAVIOR
plt.rc('text', usetex=True)
nmax = 72
ninter = 14
fit_func = lambda x, a, b : a*x+b
ppeaks1, _ = curve_fit(fit_func, list(range(0, ninter)), peaks_coef[:ninter], nan_policy = 'omit')
ppeaks2, _ = curve_fit(fit_func, list(range(ninter,nmax)), peaks_coef[ninter:nmax], nan_policy = 'omit')
pinter, _ = curve_fit(fit_func, list(range(0, nmax)), intercept[:nmax], nan_policy = 'omit')
#pdur, _ = curve_fit(fit_func, list(range(0, nmax)), duration_coef[:nmax], nan_policy = 'omit')
ppeaks = [ppeaks1]*ninter
ppeaks.extend([ppeaks2]*(nmax-ninter))
              
fig, ax = subplots_centered(2,2, nfigs = 3, figsize = (10,8))
ax1 = ax[0].twinx()
ax1.scatter(list(range(0,nmax)), peaks_coef[:nmax], label = 'Maximum wind speed')
line1 = ax1.plot(list(range(0,nmax)), fit_func(np.array(list(range(0,nmax))), *ppeaks1), '--', c = 'blue', label = 'Maximum wind speed')
ax1.set_ylabel('Speed coefficient', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.plot(list(range(0,nmax)), fit_func(np.array(list(range(0,nmax))), *ppeaks2), '--', c = 'blue')
#ax[0].scatter(list(range(0,nmax)), duration_coef, c = 'fuchsia', label = 'Duration')
#ax[0].plot(list(range(0,nmax)), fit_func(np.array(list(range(0,nmax))), *pdur[:nmax]), '--', c = 'fuchsia')
ax[0].scatter(list(range(0,nmax)), intercept[:nmax], c  = 'orange')
line2 = ax[0].plot(list(range(0,nmax)), fit_func(np.array(list(range(0,nmax))), *pinter[:nmax]), '--', c = 'orange', label = 'Intercept')
ax[0].grid()
ax[0].set_ylabel('Intercept value', color='orange')
ax[0].tick_params(axis='y', labelcolor='orange')
lines = line1 +line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left')
ax[0].set_title('Model parameters as a function of time')
ax[0].set_xlabel('Time (h)')
ax[0].set_ylabel('Parameter values')

graph_i = ninter
for i in range(1,nmax-1) :
    plot_df = pd.DataFrame()
    for j, depid in enumerate(depids_with_mld) :
        _plot_df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
        #np.argmax(r_squared)
        peaks, average, duration, mld_data, other_peaks, temp_data = get_wind_gust(_plot_df, time_diff = i)
        mld, previous_mld = [_mld_data[-1] for _mld_data in mld_data], [_mld_data[0] for _mld_data in mld_data]
        temp = [_temp_data[0] for _temp_data in temp_data]
        plot_df = pd.concat((plot_df, pd.DataFrame({'peaks':peaks, 
                                          'duration':duration, 
                                          'mld':mld, 
                                          'previous_mld':previous_mld,
                                          'temp10':temp})))
    if graph_i == i:
        graph_df = plot_df
    err = ax[1].scatter(i, np.nanmean(abs(plot_df.mld.to_numpy() 
                                       - ((ppeaks[i][0]*i + ppeaks[i][1])*plot_df.peaks.to_numpy()
                                       + pinter[0]*i + pinter[1]))),
                                       #+ (pdur[0]*i + pdur[1])*plot_df.duration.to_numpy()))),
                        c = 'red')
ax[1].legend([err], ['MLD estimation mean absolute error'])
ax[1].set_ylabel('Error (m)')
ax[1].set_xlabel('Time (h)')
ax[1].set_title('Error as a function of time')

scat = ax[2].scatter(graph_df.mld, (ppeaks[graph_i][0]*graph_i + ppeaks[graph_i][1])*graph_df.peaks.to_numpy() + pinter[0]*graph_i + pinter[1], c = graph_df.temp10) #+ plot_df.previous_mld.to_numpy()) 
ax[2].plot([0,400], [0,400], '--')
ax[2].set_xlabel('MLD')
ax[2].set_ylabel('Prediction')
fig.colorbar(scat, ax=ax[2])
ax[2].set_title("Prediction with 10m Temperature colorbar")
fig.tight_layout()

In [ ]:
fig.savefig('C:/Users/grosm/Desktop/thèse/Figures/model_parameters_and_error.pdf')

In [ ]:
###### USING CURVE FIT
mae = []
params = []
def f(X,a,b,c):
    x,y = X
    return a*x+b*y+c
def g(x, a, b, c) :
    return a*x**2 + b*x + c
bounds = [[0,-np.inf, 0],[np.inf, np.inf, 50]]
for j in range(1, 73) :
    all_mld = []
    all_temp = []
    all_var = []
    df = pd.DataFrame()
    for i, depid in enumerate(depids_with_mld) :
        _df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
        peaks, average, duration, mld_data, other_peaks, temp_data = get_wind_gust(_df, time_diff = j)
        mld, previous_mld = [_mld_data[0] for _mld_data in mld_data], [_mld_data[-1] for _mld_data in mld_data]
        temp = [_temp_data[0] for _temp_data in temp_data]
        _df = pd.DataFrame({'peaks':peaks, 
                                    'mld':mld, 
                                    'previous_mld':previous_mld,
                                    'temp10':temp})
        _df = _df.dropna(subset=['peaks', 'mld'])
        all_mld.extend([_mld_data for i,_mld_data in enumerate(mld_data) if i in _df.index])
        all_temp.extend([_temp_data for i,_temp_data in enumerate(temp_data) if i in _df.index])
        all_var.extend([np.nanvar(_temp_data) for i,_temp_data in enumerate(temp_data) if i in _df.index])
        df = pd.concat((df, _df))
    df['var_temp'] = all_var
    df.reset_index(inplace = True, drop = True)
    df = df[df.var_temp < 1500]
    df['mld_diff'] = df['mld'] - df['previous_mld']
    df = df[df.temp10.to_numpy() < 7.5]
    popt_, _ = curve_fit(g, df.peaks.to_numpy(), df.mld.to_numpy(), bounds = bounds)
    mae.append(np.mean(abs(df.mld.to_numpy() - g(df.peaks.to_numpy(), *popt_))))
    params.append(popt_)

params = np.array(params)
fig, ax = plt.subplots(1,3, figsize = (20,8))
ax[2].scatter(df.peaks.to_numpy(), df.mld.to_numpy())
ax[2].plot(list(range(0,20)), g(np.array(list(range(0,20))), *params[20]))
ax[0].plot(mae)
ax[1].plot(params[:,0])
ax[1].plot(params[:,1])
#ax[1].plot(params[:,2])



In [ ]:
### FOR LAST TIME STEP, SHOW MLD VARIATION AS A FUNCTION OF WIND SPEED AND DEPTH
temp_df = df[(df.mld > df.previous_mld) & (df.mld - df.previous_mld < df.mld * 0.5)]
wind_bins = np.arange(0, 21, 0.5)  
depth_bins = np.arange(0, 250, 10)  
temp_df["wind_bin"] = pd.cut(temp_df["peaks"], bins=wind_bins, labels=wind_bins[:-1])
temp_df["depth_bin"] = pd.cut(temp_df["previous_mld"], bins=depth_bins, labels=depth_bins[:-1])

binned_data = (temp_df.groupby(["wind_bin", "depth_bin"])["mld_diff"].mean().reset_index().dropna())
binned_data["wind_bin"] = binned_data["wind_bin"].astype(float)
binned_data["depth_bin"] = binned_data["depth_bin"].astype(float)

wind_grid = wind_bins[:-1]
depth_grid = depth_bins[:-1]
data_2d = np.full((len(depth_grid), len(wind_grid)), np.nan)

for _, row in binned_data.iterrows():
    wind_idx = np.where(wind_grid == row["wind_bin"])[0][0]
    depth_idx = np.where(depth_grid == row["depth_bin"])[0][0]
    data_2d[depth_idx, wind_idx] = row["mld_diff"]
def nanmean_kernel(values):
    return np.nanmean(values)
filtered_data_2d = generic_filter(data_2d, nanmean_kernel, size=5, mode='constant', cval=np.nan)

fig, ax = plt.subplots(figsize=(12, 8))
wind_bin_centers = (wind_bins[:-1] + wind_bins[1:]) / 2
depth_bin_centers = (depth_bins[:-1] + depth_bins[1:]) / 2
X, Y = np.meshgrid(wind_bin_centers, depth_bin_centers)
scatter = ax.scatter(X.flatten(), Y.flatten(), c=filtered_data_2d.flatten(), s=50, cmap="viridis")
#scatter = ax.imshow(filtered_data_2d, aspect = 'auto', origin = 'lower')
cbar = fig.colorbar(scatter, ax=ax, label="MLD variation (m)")
ax.set_xlabel("Wind Speed (m/s)")
ax.set_ylabel("Depth (m)")
ax.set_title("MLD Variation as a function of wind speed")
#plt.gca().invert_yaxis()
ax.grid(True)
fig.show()

In [ ]:
### PCA FOR INDEPENDANT VARIABLES

pca_model = PCA(df, ncomp=4, standardize=True, method='eig')  # Standardized PCA
factors = pca_model.factors.to_numpy()  # Principal components (scores)
loadings = pca_model.loadings.to_numpy()  # Loadings (eigenvectors)
explained_variance = pca_model.eigenvals / np.sum(pca_model.eigenvals)  # Proportion of variance explained

# 2.1 Scree Plot
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o', linestyle='--', color='b')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained')
plt.show()

# 2.2 Score Plot (PC1 vs. PC2)
plt.figure(figsize=(8, 5))
plt.scatter(factors[:, 0], factors[:, 1], alpha=0.7, edgecolors='k', label='Scores')
plt.axhline(0, color='black', linestyle='--', linewidth=0.8)
plt.axvline(0, color='black', linestyle='--', linewidth=0.8)
plt.title('Score Plot (PC1 vs. PC2)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()
plt.show()

# 2.3 Loading Plot (PC1 vs. PC2)
plt.figure(figsize=(8, 5))
for i, var in enumerate(df.columns):
    plt.arrow(0, 0, loadings[i, 0], loadings[i, 1], color='r', alpha=0.8, head_width=0.05)
    plt.text(loadings[i, 0] * 1.2, loadings[i, 1] * 1.2, var, color='g', ha='center', va='center')
plt.axhline(0, color='black', linestyle='--', linewidth=0.8)
plt.axvline(0, color='black', linestyle='--', linewidth=0.8)
plt.title('Loading Plot (PC1 vs. PC2)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid()
plt.show()

# 2.4 Biplot
plt.figure(figsize=(8, 5))
plt.scatter(factors[:, 0], factors[:, 1], alpha=0.7, edgecolors='k', label='Scores')
for i, var in enumerate(df.columns):
    plt.arrow(0, 0, loadings[i, 0], loadings[i, 1], color='r', alpha=0.8, head_width=0.05)
    plt.text(loadings[i, 0] * 1.2, loadings[i, 1] * 1.2, var, color='g', ha='center', va='center')
plt.axhline(0, color='black', linestyle='--', linewidth=0.8)
plt.axvline(0, color='black', linestyle='--', linewidth=0.8)
plt.title('Biplot (Scores and Loadings)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()
plt.show()

# 2.5 Cumulative Variance Explained Plot
cumulative_variance = np.cumsum(explained_variance)
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--', color='g')
plt.title('Cumulative Variance Explained')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Proportion of Variance Explained')
plt.axhline(y=0.9, color='r', linestyle='--', label='90% Variance Explained')
plt.legend()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize = (7,7))
df['init_mld'] = df.previous_mld < 30
sns.kdeplot(df, x='peaks', y='mld', hue = 'init_mld', ax = ax)
ax.set_ylim(-10, 400)